###**A Recommender Chatbot for Name-search and Phone-model search**

In [21]:
import sys
import os
import pandas as pd
import numpy as np
                                                          
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
from nltk.tokenize import word_tokenize                                                              
from nltk.tokenize import sent_tokenize                                                        
nltk.download('punkt')
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
sw = stopwords.words('english')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
# Functions for RECOMMENDATION BASED ON THE ENTITY VALUES

# For NAME-SEARCHING

def name_attr_info():
    op_nm = pd.read_excel('finalnames.xlsx')
    #print(op_nm.head(3))
    print('The child is a boy or girl ?','\n')
    gen = input().lower()
    if gen in op_nm.Gender.values :
        catalog = op_nm.loc[op_nm['Gender'] == gen]
    else:
        print('Give correct option(boy/girl. ')
    print('The Zodiac names : ', op_nm['Zodiac'].unique())
    print('What is the Zodiac-sign ?', '\n')
    zodiac = input().capitalize()

    if zodiac in op_nm['Zodiac'].values:
        catalog = catalog.loc[catalog['Zodiac'] == zodiac]
        print('Your options of month for given zodiac ', catalog['Month'].unique())
    else:
        print('Check your input. ')

    print('What is the birth month ?','\n')
    month =input().capitalize()

    if month in op_nm['Month'].values:
        catalog = catalog.loc[catalog['Month'] == month]
        print('You have choices for the first letter : ', catalog['First_letter'].unique())
    else:
        print('Check your input. ', '\n', 'You have choices :', catalog['Month'].unique())

    print('Which First-letter should the name start ?','\n')
   
    first = input().lower()
    if first in op_nm['First_letter'].values:
        catalog = catalog.loc[catalog['First_letter'] == first]
        match = catalog['Name'].unique()
        print('Best choice for name : ')
        print(match)
    else:
        print('Check your input. ', '\n', 'You have choices :', catalog['First_letter'])

# For PHONE-SEARCHING    

def phone_attr_info():
    op_ph = pd.read_excel('finalphones.xlsx')
    #print(op_ph.head(3))
    print('Which brand phone () ?','\n')
    print(op_ph.Brand.unique())
    brand = input().capitalize()
    if brand in op_ph.Brand.values :
        catalog2 = op_ph.loc[op_ph['Brand'] == brand]
    else:
        print('Give correct option ')

    print('Your options for camera : ', catalog2.Camera.unique())
    print('What is the Camera(MP) ?', '\n')
    cam = int(input())

    if cam in op_ph['Camera'].values:
        catalog2 = catalog2.loc[catalog2['Camera'] == cam]
    else:
        print('Check your input. ')
    
    print('What is the Price ?','\n')
    price =int(input())
    print('Your options for price : ', catalog2.Price.unique())
    if price in op_ph['Price'].values:
        catalog2 = catalog2.loc[catalog2['Price'] == price]
    else:
        print('Check your input. ')
    print('You have choices of RAM ', catalog2['RAM'].unique())
    print('How much RAM(GB) ?','\n')
    ram = int(input())

    if ram in op_ph['RAM'].values:
#        match2 = catalog2.loc[catalog2['RAM'] == ram].values
        match2 = catalog2['Model'].unique()
        print('The best choice for model of phone ')
        print(match2)

In [23]:

## function to lemmatize and tokenize 
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [26]:
  #LOAD THE COMBINED UTTERANCES CSV FILE

df  = pd.read_csv('all_intents.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace =True)

 #GET CLEANED SENTENCE AND ADD TO NEW COLUMN 'Utterences_cleaned'   

df['Utterences_cleaned'] = df.Intent.apply(lemmatize_text)                 ## tokenizing and lemmatizing the sentences 
df['Utterences_cleaned'] = df['Utterences_cleaned'].apply(lambda x: [item for item in x if item not in sw])    ## remove stop words
  
  #PREPARE THE FEATURES 'X'

  X = df['Utterences_cleaned']                              # featureset

  #PREPARE THE LABEL 'y'

  y = df['Intent class']                       # target

  #IMPLEMENT WORD EMBEDDING 

X =X.apply(lambda x: " ".join(x) )
count_vec = CountVectorizer()
vec_df = pd.DataFrame(count_vec.fit_transform(X).toarray(), columns =count_vec.get_feature_names() )
X = vec_df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=167)

  #CHOOSE THE CLASSIFICATION MODEL
 
knn = KNeighborsClassifier(n_neighbors=3)
  
  #TRAIN THE MODEL ON TRAIN DATA

knn.fit(X_train, y_train)
  
  #PREDICT THE MODEL ON TEST DATA

y_kn = knn.predict(X_test)
  
knn_accuracy = accuracy_score(y_kn, y_test)

cor = input('How this bot can assist you ?  ::  ')
cor_cl = lemmatize_text(cor)
cor_cl = [w for w in cor_cl if w not in sw]

df_cor = pd.DataFrame(count_vec.transform(cor_cl).toarray(), columns = count_vec.get_feature_names())

  #PREDICT THE INTENT

int_predict = knn.predict(df_cor)               # predict the user intention from utterance
int_proba = knn.predict_proba(df_cor)   # probability prediction from decision tree model

x1 = np.sum(int_proba, axis = 0)                  # sum probability for wrt class/column
x1 = list(x1)                                                              # make a list

input_dict = {'name': x1[0], 'phone': x1[1]}
Search_intent = max(input_dict, key=input_dict.get)                   # get the intent based on probability score
print('The classified intent : ', Search_intent)

if Search_intent=='name':
#    print('The Best match found is ', '\n')
    name_attr_info()                                                        # calling the name-searcing function

elif Search_intent=='phone':
    phone_attr_info()                                                      # calling the phone searching function

How this bot can assist you ?  ::  name
The classified intent :  name
The child is a boy or girl ? 

girl
The Zodiac names :  ['Aries' 'Taurus' 'Gemini' 'Cancer' 'Leo' 'Virgo' 'Libra' 'Scorpius'
 'Sagittarius' 'Capricornus' 'Aquarius' 'Pisces']
What is the Zodiac-sign ? 

leo
Your options of month for given zodiac  ['July' 'August']
What is the birth month ? 

july
You have choices for the first letter :  ['ma' 'me' 'mo']
Which First-letter should the name start ? 

ma
Best choice for name : 
['Mamta']


### **Test Name-Search case only**

In [ ]:
    op_nm = pd.read_excel('finalnames.xlsx')
    #print(op_nm.head(3))
    print('The child is a boy or girl ?','\n')
    gen = input().lower()
    if gen in op_nm.Gender.values :
        catalog = op_nm.loc[op_nm['Gender'] == gen]
    else:
        print('Give correct option(boy/girl. ')
    
    print('What is the Zodiac-sign ?', '\n')
    zodiac = input().capitalize()

    if zodiac in op_nm['Zodiac'].values:
        catalog = catalog.loc[catalog['Zodiac'] == zodiac]
        print('Your options of month for given zodiac ', catalog['Month'].unique())
    else:
        print('Check your input. ')

    print('What is the birth month ?','\n')
    month =input().capitalize()

    if month in op_nm['Month'].values:
        catalog = catalog.loc[catalog['Month'] == month]
        print('You have choices : ', catalog['First_letter'].unique())
    else:
        print('Check your input. ', '\n', 'You have choices :', catalog['Month'].unique())

    print('Which First-letter should the name start ?','\n')
    first = input().lower()

    if first in op_nm['First_letter'].values:
        catalog = catalog.loc[catalog['First_letter'] == first]
        match = catalog['Name'].unique()
        print('Best choice for name : ')
        print(match)
    else:
        print('Check your input. ', '\n', 'You have choices :', catalog['First_letter'])

### **Test Phone-Search case only**

In [ ]:
    op_ph = pd.read_excel('finalphones.xlsx')
    #print(op_ph.head(3))
    print('Which brand phone () ?','\n')
    print(op_ph.Brand.unique())
    brand = input().capitalize()
    if brand in op_ph.Brand.values :
        catalog2 = op_ph.loc[op_ph['Brand'] == brand]
    else:
        print('Give correct option ')

    print('Your options for camera : ', catalog2.Camera.unique())
    print('What is the Camera(MP) ?', '\n')
    cam = int(input())

    if cam in op_ph['Camera'].values:
        catalog2 = catalog2.loc[catalog2['Camera'] == cam]
    else:
        print('Check your input. ')
    
    print('What is the Price ?','\n')
    print('Your options for price : ', catalog2.Price.unique())
    price =int(input())

    if price in op_ph['Price'].values:
        catalog2 = catalog2.loc[catalog2['Price'] == price]
    else:
        print('Check your input. ')
    print('You have choices of RAM ', catalog2['RAM'].unique())
    print('How much RAM(GB) ?','\n')
    ram = int(input())

    if ram in op_ph['RAM'].values:
        #catalog = catalog.loc[catalog['First_letter'] == first]
        #match = catalog['Output'].unique()
        match2 = catalog2.loc[catalog2['RAM'] == ram].values
        print('The best choice for model of phone ')
        print(match2)

Which brand phone () ? 

['Apple' 'Samsung' 'Xiaomi' 'Realme' 'Oneplus' 'Google']
Apple
Your options for camera :  [12]
What is the Camera(MP) ? 

12
What is the Price ? 

Your options for price :  [129900 119900  69900  93599  83799  50999  47900  39000]
119900
You have choices of RAM  [6]
How much RAM(GB) ? 

6
The best choice for model of phone 
[['Apple' 12 119900 6 'iPhone 12 Pro']]
